# Restarting simulation from a GSD file

In [2]:
## -----Import essential packages
from __future__ import division
import hoomd
import hoomd.md

#-----Define important variables

t_max = 8.0

#-----Start from a gsd file
hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Select the trayectory of the center of mass of particles

system = hoomd.init.read_gsd("T_CM_8.0_restarted_rigid.gsd",frame = -1)

#-----Define each rigid body type in the local coordinate system of the body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Instruct the rigid constraint to create the constituent particles

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.cell()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define interaction with species in the binary mixture

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Select an standard integrator

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define group

mesogens = hoomd.group.rigid_center();

#----Integrate using NVT

npt = hoomd.md.integrate.npt(group = mesogens, kT = t_max, tau = 1.0, tauP = 0.5, P = 0.1);

#-----Write output and Run the Simulation

log_file = "T_" + str(t_max) + "_stabilized_trayectory.log"
gsd_file = "T_" + str(t_max) + "_stabilized_trayectory.gsd" 
meso_gsd_file = "T_CM_" + str(t_max) + "stabilized_trayectory_.gsd"
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e4,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e4,
               group = hoomd.group.all(),
               overwrite=True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e4,
               group = mesogens,
               overwrite=True); 
hoomd.run(1e6)

notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
notice(2): Group "rigid_center" created containing 1000 particles
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 1130633 / 2130000 | TPS 63.2045 | ETA 04:23:31
Time 00:00:20 | Step 1131263 / 2130000 | TPS 62.745 | ETA 04:25:17
Time 00:00:30 | Step 1131961 / 2130000 | TPS 69.7246 | ETA 03:58:34
Time 00:00:40 | Step 1132483 / 2130000 | TPS 52.029 | ETA 05:19:32
Time 00:00:50 | Step 1133005 / 2130000 | TPS 52.0789 | ETA 05:19:03
Time 00:01:00 | Step 1133533 / 2130000 | TPS 52.7706 | ETA 05:14:42
Time 00:01:10 | Step 1134053 / 2130000 | TPS 51.8736 | ETA 05:19:59
Time 00:01:20 | Step 1134547 / 2130000 | TPS 49.2971 | ETA 05:36:32
Time 00:01:

**ERROR**: Error allocating aligned memory


RuntimeError: Error allocating GPUArray.